In [1]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.8 MB/s 
     |████████████████████████████████| 163 kB 55.5 MB/s 
     |████████████████████████████████| 7.6 MB 40.6 MB/s 


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.2 MB/s 
     |████████████████████████████████| 95 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 50.2 MB/s 
     |████████████████████████████████| 115 kB 59.1 MB/s 
     |████████████████████████████████| 127 kB 69.3 MB/s 
     |████████████████████████████████| 115 kB 44.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
import json
import os

In [19]:
os.getcwd()

'/content'

In [32]:
valid_focus = json.load(open('valid_focus.json'))

In [34]:
for i, line in enumerate(valid_focus['data']):
  for key in line.keys():
    print (key, line[key])
  break

dialogID 4YZMOWMPX9UC
persona ['I would like to visit the Nazareth House again.', 'I love Benevolent institutions.', 'I am interested in History.', 'I have curiosity about the Description of this place.', 'I would like to know when it was Built.']
landmark_link 'https://en.wikipedia.org/wiki/Nazareth_House,_Wynnum
knowledge ['Nazareth House is a heritage-listed benevolent institution at 272 Wynnum North Road, Wynnum, City of Brisbane, Queensland, Australia. It was built from 1924 to 1939. It was added to the Queensland Heritage Register on 2 April 2002.', 'Nazareth House, situated on Tingal Hill, Wynnum, was designed by Brisbane architectural firm, Hennessy, Hennessy, Keesing & Co and JP Donoghue and built by George Turner. Nazareth House was officially opened by Archbishop James Duhig in 1925 as part of the charitable institution established on the site by the Poor Sisters of Nazareth.', 'A Catholic presence in Wynnum was prompted by the establishment of the area as a popular seaside 

In [ ]:
# persona_grounding [True, False, False, False]
# knowledge_answer_index 0

In [ ]:
import json
import logging
import os
import torch
from transformers import cached_path

logger = logging.getLogger(__file__)

def get_dataset_only_train_dev(tokenizer, train_dataset_path, train_dataset_cache, dev_dataset_path, dev_dataset_cache):

    def tokenize(obj):
        if isinstance(obj, str):
            return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(obj))
        if isinstance(obj, dict):
            return dict((n, tokenize(o)) for n, o in obj.items())
        return list(tokenize(o) for o in obj)

    train_dataset_cache = train_dataset_cache + '_train_focus_' + type(tokenizer).__name__
    dev_dataset_cache = dev_dataset_cache + '_dev_focus_' + type(tokenizer).__name__

    # train_dataset_cache = ""
    # dev_dataset_cache = ""
    if train_dataset_cache and os.path.isfile(train_dataset_cache):
        logger.info("Load tokenized dataset from cache at %s", train_dataset_cache)
        train_dataset = torch.load(train_dataset_cache)
        dev_dataset = torch.load(dev_dataset_cache)
        all_dataset = dict()
        all_dataset["train"] = train_dataset["train"]
        all_dataset["valid"] = dev_dataset["valid"]
    else:
        logger.info("Process dataset from %s", train_dataset_path)
        plan_file_train = cached_path(train_dataset_path)
        plan_file_dev = cached_path(dev_dataset_path)
        file_dict = {"train": plan_file_train, "valid": plan_file_dev}
        all_dataset = dict()

        # import pdb; pdb.set_trace()

        for name, file in file_dict.items():
            # import pdb; pdb.set_trace()
            with open(file, "r", encoding="utf-8") as f:

                # import pdb;
                # pdb.set_trace()
                dataset_enc = dict()
                dataset_enc[name] = list()
                for line in f:
                    dialogue = json.loads(line)

                    # import pdb;
                    # pdb.set_trace()
                    ID = dialogue["dialogID"]
                    persona = dialogue["persona"]
                    knowledge = dialogue["knowledge"]
                    utterance = dialogue["utterance"]
                    new_dialogue = dict()
                    new_dialogue["utterance"] = list()
                    for i, utt in enumerate(utterance):
                        # import pdb; pdb.set_trace()
                        key = "dialogue" + str(i+1)
                        dial = utt[key]
                        dial_new = dict()
                        persona_can = utt["persona_candidate"]
                        if len(persona_can) > 5:
                            persona_can = persona_can[:5]
                        persona_ground = utt["persona_grounding"]
                        if len(persona_ground) > 5:
                            persona_ground = persona_ground[:5]
                        knowledge_can = utt["knowledge_candidates"]
                        knowledge_answer = utt["knowledge_answer_index"]
                        filtered_triple_can = utt['filtered_triple_candidates']
                        ners = utt['entities_in_kg']

                        dial_enc = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence.strip())) for sentence in dial]
                        persona_ground_enc = [1 if item==True else 0 for item in persona_ground]
                        knowledge_can_enc = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence.strip())) for sentence in knowledge_can]
                        persona_can_enc = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence.strip())) for sentence in persona_can]

                        dial_new["dialog"] = dial_enc
                        dial_new["persona_grounding"] = persona_ground_enc
                        dial_new["persona_candidates"] = persona_can_enc
                        dial_new["knowledge_candidates"] = knowledge_can_enc
                        dial_new["knowledge_answer_index"] = knowledge_answer
                        dial_new['filtered_triple_candidates'] = filtered_triple_can_enc
                        dial_new['ner_knowledge_candidates'] = ners_concat_ent
                        new_dialogue["utterance"].append(dial_new)

                        # import pdb;
                        # pdb.set_trace()
                    persona_enc = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence.strip())) for sentence in persona]
                    knowledge_enc = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence.strip())) for sentence in knowledge]
                    new_dialogue["persona"] = persona_enc
                    new_dialogue["knowledge"] = knowledge_enc
                    new_dialogue["dialogID"] = ID
                    dataset_enc[name].append(new_dialogue)
            logger.info("Tokenize and encode the dataset")
            dataset = dataset_enc
            all_dataset[name] = dataset_enc[name]
            if name == 'train':
                torch.save(dataset, train_dataset_cache)
            else:
                torch.save(dataset, dev_dataset_cache)

    return all_dataset



def get_dataset_only_test(tokenizer, dataset_path, dataset_cache, without_rel, line_proc):

    def tokenize(obj):
        if isinstance(obj, str):
            return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(obj))
        if isinstance(obj, dict):
            return dict((n, tokenize(o)) for n, o in obj.items())
        return list(tokenize(o) for o in obj)

    dataset_cache = dataset_cache + '_test_focus_' + type(tokenizer).__name__

    if dataset_cache and os.path.isfile(dataset_cache):
        logger.info("Load tokenized dataset from cache at %s", dataset_cache)
        dataset = torch.load(dataset_cache)
    else:
        logger.info("Process dataset from %s", dataset_path)
        plan_file = cached_path(dataset_path)
        with open(plan_file, "r", encoding="utf-8") as f:
            if not line_proc:
                dataset = json.loads(f.read())
            else:
                lines = f.readlines()
                dataset = [json.loads(line) for line in lines]
            # import pdb; pdb.set_trace()
            dataset_enc = dict()
            dataset_enc["test"] = list()
            for dialogue in dataset:
                # import pdb;
                # pdb.set_trace()
                ID = dialogue["dialogID"]
                persona = dialogue.get("persona")
                knowledge = dialogue["knowledge"]
                utterance = dialogue["utterance"]
                new_dialogue = dict()
                new_dialogue["utterance"] = list()
                for i, utt in enumerate(utterance):
                    key = "dialogue" + str(i+1)
                    dial = utt[key]
                    dial_new = dict()
                    persona_can = utt["persona_candidate"]
                    if len(persona_can) > 5:
                        persona_can = persona_can[:5]
                    persona_ground = utt["persona_grounding"]
                    if len(persona_ground) > 5:
                        persona_ground = persona_ground[:5]
                    knowledge_can = utt["knowledge_candidates"]
                    knowledge_answer = utt["knowledge_answer_index"]
                    filtered_triple_can = utt['filtered_triple_candidates']
                    ners = utt['entities_in_kg']
                    # import pdb; pdb.set_trace()

                    dial_enc = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence.strip())) for sentence in dial]
                    persona_ground_enc = [1 if item==True else 0 for item in persona_ground]
                    knowledge_can_enc = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence.strip())) for sentence in knowledge_can]
                    persona_can_enc = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence.strip())) for sentence in persona_can]
                    # import pdb; pdb.set_trace()

                    # Not needed for v5, v6
                    # filtered_triple_can = [triple_list[0] for triple_list in filtered_triple_can ]
                    filtered_concat_triple_can = []

                    filtered_triple_lists = []

                    # assert len(filtered_triple_can) == 0 or len(filtered_triple_can) == 1
                    for cans in filtered_triple_can:
                        subjects = []
                        relations = []
                        objects = []
                        for can in cans:
                            splits = can.split(" [SEP] ")
                            if len(splits) == 3:
                                subjects.append(splits[0])
                                relations.append(splits[1])
                                objects.append(splits[2])
                        if not without_rel:
                            sents = [subjects[i] + " " + relations[i] + " " + objects[i] for i in range(len(subjects))]
                            triples = [[subjects[i] + " [SEP] " + relations[i] + " [SEP] " + objects[i]] for i in range(len(subjects))]
                        else:
                            sents = [subjects[i] + " " + objects[i] for i in range(len(subjects))]
                            triples = [subjects[i] + " [SEP] " + objects[i] for i in range(len(subjects))]
                        filtered_concat_triple_can.append(" ".join(sents))
                        filtered_triple_lists.append(" [TRIPLE_SEP] ".join(triples))
                    filtered_triple_can_enc = [tokenizer.convert_tokens_to_ids([tok for tok in tokenizer.tokenize(
                        sentence.strip() if sentence else "")]) for sentence in filtered_concat_triple_can]
                    filtered_triple_lists_enc = [tokenizer.convert_tokens_to_ids([tok for tok in tokenizer.tokenize(
                        triple.lower().strip() if triple else "")]) for triple in filtered_triple_lists]
                    # import pdb; pdb.set_trace()

                    ners = [ent[0] for ent in ners]
                    ners_concat_ent = []
                    for ent in ners:
                        rep = ""
                        # TODO : value type wise filtering
                        rep += " ".join(ent.keys())
                        ners_concat_ent.append(rep)
                    ners_concat_ent = [tokenizer.convert_tokens_to_ids([tok for tok in tokenizer.tokenize(
                        sentence.strip() if sentence else "")]) for sentence in ners_concat_ent]
                    # import pdb; pdb.set_trace()

                    dial_new["dialog"] = dial_enc
                    dial_new["persona_candidates"] = persona_can_enc
                    dial_new["persona_grounding"] = persona_ground_enc
                    dial_new["knowledge_candidates"] = knowledge_can_enc
                    dial_new["knowledge_answer_index"] = knowledge_answer
                    dial_new['filtered_triple_candidates'] = filtered_triple_can_enc
                    dial_new['filtered_triple_candidates_raw'] = filtered_triple_lists_enc
                    dial_new['ner_knowledge_candidates'] = ners_concat_ent
                    new_dialogue["utterance"].append(dial_new)
                persona_enc = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence.strip())) for sentence in (persona if persona else persona_can)]
                knowledge_enc = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence.strip())) for sentence in knowledge]
                new_dialogue["persona"] = persona_enc
                new_dialogue["knowledge"] = knowledge_enc
                new_dialogue["dialogID"] = ID
                dataset_enc["test"].append(new_dialogue)
                # import pdb; pdb.set_trace()
        logger.info("Tokenize and encode the dataset")
        dataset = dataset_enc
        torch.save(dataset, dataset_cache)

    return dataset



class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self



def make_focus_logdir(dir_name: str):
    logdir = os.path.join(
        './models', dir_name
    )
    return 


In [38]:
from transformers import BertTokenizer

In [54]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [55]:
tokenizer

PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [56]:
len(tokenizer)

30522

In [57]:
tokenizer.add_tokens('<hello.')

1

In [58]:
len(tokenizer)

30523

In [60]:
tokenizer.convert_ids_to_tokens(len(tokenizer)-1)

'<hello.'

In [ ]:
model.resize_embedding(len(tokenizer))

In [ ]:
b#(c) 2021 NCSOFT Corporation & Korea University. All rights reserved.
import logging
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, BCEWithLogitsLoss
from transformers import GPT2Model, GPT2PreTrainedModel, GPT2LMHeadModel
from transformers import BartModel, BartPretrainedModel, BartForConditionalGeneration
from torch.nn import Sigmoid, Softmax


logger = logging.getLogger(__name__)


class ConcatSummary(nn.Module):
    def __init__(self, emb_dim=768):
        super().__init__()
        self.dropout = nn.Dropout(0.1)
        self.summary = nn.Linear(emb_dim * 7, 1)  # hiddensize, numclasses
    def forward(self, output):
        dropout_pooled_output = self.dropout(output)
        logits = self.summary(dropout_pooled_output)
        return logits

class Summary(nn.Module):
    def __init__(self, emb_dim=768):
        super().__init__()
        self.dropout = nn.Dropout(0.1)
        self.summary = nn.Linear(emb_dim , 1)  # hiddensize, numclasses
    def forward(self, output):
        dropout_pooled_output = self.dropout(output)
        logits = self.summary(dropout_pooled_output)
        return logits



class GPT2PK_ctxt(GPT2LMHeadModel):
    _keys_to_ignore_on_load_missing = [r"h\.\d+\.attn\.masked_bias", r"lm_head\.weight"]

    def __init__(self, config):
        super().__init__(config)
        self.transformer = GPT2Model(config)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.concat_summary = ConcatSummary(emb_dim=config.n_embd)
        self.summary = Summary(emb_dim=config.n_embd)
        self.attn1 = nn.Linear(config.n_embd, 5)
        self.attn2 = nn.Linear(5, config.n_embd)# Selected knowledge 개수만
        self.max_position = config.n_positions
        #self.paragraph_sum = nn.Linear(config.n_embd, 1)
        self.init_weights()

    def forward(
            self,
            input_ids=None,
            input_eos=None,
            token_type_ids=None,
            only_dial_input_ids=None,
            only_dial_token_type_ids=None,
            persona_input_ids=None,
            knowledge_input_ids=None,
            persona_can_idx=None,
            persona_grounding=None,
            knowledge_can_idx=None,
            knowledge_grounding=None,
            tot_knowledge=None,
            tot_knowledge_token_ids=None,
            tot_knowledge_eos=None,
            training=None,
            mc_token_ids=None):

        persona = 50259
        knowledge = 50260
        padding = 50261
        bos = 50256
        device = input_ids.get_device()

        persona_tensor = torch.tensor([persona]).cuda(device)
        knowledge_tensor = torch.tensor([knowledge]).cuda(device)
        bos_tensor = torch.tensor([bos]).cuda(device)
        outputs = tuple()
        dynamic_lm_logits = None
        persona_logits = None
        knowledge_logits = None
        lm_labels=None

        if input_eos is not None:
            lm_hidden_states = self.transformer(input_ids=input_ids, token_type_ids=token_type_ids)['last_hidden_state']
            batch, seq_len, embdim = lm_hidden_states.size()
            lm_hidden_states_eos_list = []
            for i in range(batch):
                lm_hidden_states_batch = lm_hidden_states[i]
                lm_eos_batch = input_eos[i]
                lm_hidden_states_eos = torch.index_select(lm_hidden_states_batch, -2, lm_eos_batch)
                lm_hidden_states_eos_list.append(lm_hidden_states_eos)
            lm_eos_rep = torch.stack(lm_hidden_states_eos_list)
            #print("lm eos rep: ", lm_eos_rep.size()) #batch, 1, embdim

            tot_knowledge_hidden_states = self.transformer(input_ids = tot_knowledge, token_type_ids=tot_knowledge_token_ids)['last_hidden_state']
            #print("tot knowledge: ", tot_knowledge_hidden_states.size()) #batch, 5(# paragraph), seqlen, embdim
            tot_knowledge_eos_list = []
            for i in range(batch):
                tot_knowledge_hidden_states_batch = tot_knowledge_hidden_states[i]
                tot_knowledge_eos_batch = tot_knowledge_eos[i]
                #print("tot_knowledge_hid batch: ", tot_knowledge_hidden_states_batch.size(), tot_knowledge_eos_batch.size()) #5, seqlen, embdim / 5
                tot_knowledge_eos_list_batch = []
                for j in range(5):
                    tot_knowledge_eos_token = torch.index_select(tot_knowledge_hidden_states_batch[j], -2, tot_knowledge_eos_batch[j])
                    tot_knowledge_eos_list_batch.append(tot_knowledge_eos_token.squeeze())
                tot_knowledge_eos_batch_rep = torch.stack(tot_knowledge_eos_list_batch)
                tot_knowledge_eos_list.append(tot_knowledge_eos_batch_rep)
            tot_knowledge_eos_final = torch.stack(tot_knowledge_eos_list)
            knowledge_inctxt_attn = self.attn1(tot_knowledge_eos_final)
            knowledge_inctxt_eos_rep = self.attn2(knowledge_inctxt_attn)
            inctxt_states = torch.cat((lm_eos_rep, knowledge_inctxt_eos_rep), dim=1).type_as(input_ids)

            sigmoid = Sigmoid()
            #persona candidates
            num_persona_can = 5
            if persona_input_ids is not None:
                persona_emb = self.transformer(input_ids=persona_input_ids)['last_hidden_state']
                if persona_can_idx is not None:
                    persona_list = []
                    for batch_i in range(batch):
                        inctxt_eos_batch = inctxt_states[batch_i] #6, 768
                        persona_emb_batch = persona_emb[batch_i]
                        persona_can_idx_batch = persona_can_idx[batch_i]
                        persona_batch_list = []
                        for i in range(num_persona_can):
                            persona_selected = torch.index_select(persona_emb_batch[i], 0, persona_can_idx_batch[i])
                            final_rep_persona = torch.cat([inctxt_eos_batch.type_as(lm_eos_rep), persona_selected.type_as(lm_eos_rep)], dim=0) #7,768
                            persona_batch_list.append(final_rep_persona)
                        persona_batch_list = torch.stack(persona_batch_list)
                        persona_list.append(persona_batch_list)
                    persona_rep = torch.stack(persona_list).view(batch*num_persona_can, -1)
                    persona_logits = self.concat_summary(persona_rep).view(batch, -1)
                    outputs = (persona_logits, )

                    persona_pred_sigmoid = sigmoid(persona_logits)
                    persona_pred_sigmoid = (persona_pred_sigmoid > 0.5).float()
                    all_persona_pred = []
                    selected_persona_idx = list()
                    for batch_idx, persona_batch in enumerate(torch.eq(persona_pred_sigmoid, 1)):
                        batch_list_idx = list()
                        batch_list = list()
                        for i, can in enumerate(persona_batch):
                            if can == True:
                                batch_list_idx.append(can)
                                persona_selected_now = persona_input_ids[batch_idx][i]
                                mask_persona = torch.ne(persona_selected_now, padding)
                                persona_selected_now = torch.masked_select(persona_selected_now, mask_persona)
                                batch_list.append(persona_selected_now[:-2])
                        all_persona_pred.append(batch_list)
                        selected_persona_idx.append(batch_list_idx)


            #knowledge candidates
            num_knowledge_can = 10
            if knowledge_input_ids is not None:
                knowledge_emb = self.transformer(input_ids=knowledge_input_ids)['last_hidden_state']
                if knowledge_can_idx is not None:
                    knowledge_list = []
                    for batch_i in range(batch):
                        inctxt_eos_batch = inctxt_states[batch_i]
                        knowledge_emb_batch = knowledge_emb[batch_i]
                        knowledge_can_idx_batch = knowledge_can_idx[batch_i]
                        knowledge_batch_list = []
                        for i in range(num_knowledge_can):
                            knowledge_selected = torch.index_select(knowledge_emb_batch[i], 0, knowledge_can_idx_batch[i])
                            final_rep_knowledge = torch.cat([inctxt_eos_batch.type_as(lm_eos_rep), knowledge_selected.type_as(lm_eos_rep)], dim=0)
                            knowledge_batch_list.append(final_rep_knowledge)
                        knowledge_batch_list = torch.stack(knowledge_batch_list)
                        knowledge_list.append(knowledge_batch_list)
                    knowledge_rep = torch.stack(knowledge_list).view(batch*num_knowledge_can, -1)
                    knowledge_logits = self.concat_summary(knowledge_rep).view(batch, -1)
                    outputs = (knowledge_logits,) + outputs
                    softmax = Softmax(dim=-1)
                    knowledge_softmax = softmax(knowledge_logits)
                    _, k_index_1 = torch.topk(knowledge_softmax, k=1, dim=-1)
                    all_knowledge_pred = []
                    for batch_i in range(batch):
                        knowledge_pred_idx = k_index_1[batch_i]
                        knowledge_pred = knowledge_input_ids[batch_i][knowledge_pred_idx]
                        mask_knowledge = torch.ne(knowledge_pred, padding)
                        knowledge_pred = torch.masked_select(knowledge_pred, mask_knowledge)
                        knowledge_pred = knowledge_pred[1:-2] #delete bos, knowledge_st, eos
                        if knowledge_pred.size()[0] > 150:
                            knowledge_pred = knowledge_pred[:150]
                        all_knowledge_pred.append(knowledge_pred)


            final_input_list = []
            final_input_tti_list = []
            final_lm_label_list = []
            for batch_i in range(batch):
                only_dial_input_ids_batch = only_dial_input_ids[batch_i]
                only_dial_token_type_ids_batch = only_dial_token_type_ids[batch_i]
                mask_only_dial_input_ids_batch = torch.ne(only_dial_input_ids_batch, padding)
                mask_only_dial_tti_batch = torch.ne(only_dial_token_type_ids_batch, padding)
                only_dial_input_ids_batch = torch.masked_select(only_dial_input_ids_batch, mask_only_dial_input_ids_batch)
                only_dial_token_type_ids_batch = torch.masked_select(only_dial_token_type_ids_batch, mask_only_dial_tti_batch)
                if len(all_persona_pred[batch_i])>0:
                    concat_persona = torch.cat(all_persona_pred[batch_i], dim=-1)
                    new_persona = torch.cat([persona_tensor, concat_persona], dim=-1)
                    new_persona_tti = torch.tensor([persona] * (new_persona.size()[0])).cuda(device)
                else:
                    new_persona = None
                    new_persona_tti = None

                new_knowledge = torch.cat([knowledge_tensor, all_knowledge_pred[batch_i]], dim=-1)
                new_knowledge_tti = torch.tensor([knowledge] * (new_knowledge.size()[0])).cuda(device)

                if new_persona is not None:
                    new_input = torch.cat([bos_tensor, new_knowledge, new_persona, only_dial_input_ids_batch], dim=-1)
                    new_input_tti = torch.cat([knowledge_tensor, new_knowledge_tti, new_persona_tti, only_dial_token_type_ids_batch], dim=-1)
                else:
                    new_input = torch.cat([bos_tensor, new_knowledge, only_dial_input_ids_batch], dim=-1)
                    new_input_tti = torch.cat([knowledge_tensor, new_knowledge_tti, only_dial_token_type_ids_batch], dim=-1)

                new_input_size = new_input.size()[0]
                new_lm_label = torch.cat([torch.tensor([-100] * (new_input_size-(only_dial_input_ids_batch.size()[0])+1)).cuda(device), only_dial_input_ids_batch[1:]], dim=-1)
                assert new_input.size() == new_input_tti.size() == new_lm_label.size()
                if new_input_size < int(self.max_position):
                    padding_size = int(self.max_position)-new_input_size
                    add_padding = torch.tensor([padding]*padding_size).cuda(device)
                    add_lm_padding = torch.tensor([-100]*padding_size).cuda(device)
                    final_input = torch.cat([new_input, add_padding], dim=-1)
                    final_tti_input = torch.cat([new_input_tti, add_padding], dim=-1)
                    final_lm_label = torch.cat([new_lm_label, add_lm_padding], dim=-1)
                final_input_list.append(final_input)
                final_input_tti_list.append(final_tti_input)
                final_lm_label_list.append(final_lm_label)
            input_ids = torch.stack(final_input_list)
            token_type_ids = torch.stack(final_input_tti_list)
            lm_labels = torch.stack(final_lm_label_list)


        dynamic_lm_hidden_states = self.transformer(input_ids=input_ids, token_type_ids=token_type_ids)['last_hidden_state']
        if dynamic_lm_hidden_states is not None:
            dynamic_lm_logits = self.lm_head(dynamic_lm_hidden_states)
            outputs = (dynamic_lm_logits,) + outputs

        if persona_grounding is not None:
            loss_fct = BCEWithLogitsLoss()
            persona_loss = loss_fct(persona_logits.view(batch, -1), persona_grounding.type_as(persona_logits))
            outputs = (persona_loss,) + outputs

        if knowledge_grounding is not None:
            loss_fct = CrossEntropyLoss()
            knowledge_loss = loss_fct(knowledge_logits.view(batch, -1), knowledge_grounding)
            outputs = (knowledge_loss,) + outputs

        if training is not True:
            outputs = (lm_labels,) + outputs
            lm_labels = None

        if lm_labels is not None:
            shift_logits = dynamic_lm_logits[..., :-1, :].contiguous()
            shift_labels = lm_labels[..., 1:].contiguous()
            loss_fct = CrossEntropyLoss(ignore_index=-100)
            lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            outputs = (lm_loss,) + outputs

        return outputs  # (lm_loss-training), (lm_label-validation), (knowledge_loss), (persona_loss), dynamic_lm_logits, knowledge_logits, persona_logits, presents, (all hidden_states), (attentions)



class BARTPK_ctxt(BartForConditionalGeneration):
    _keys_to_ignore_on_load_missing = [r"h\.\d+\.attn\.masked_bias", r"lm_head\.weight"]

    def __init__(self, config):
        super().__init__(config)

        self.model = BartModel(config)
        self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False) # 768 x 32000 -> argmax -> idx -> token
        self.concat_summary = ConcatSummary(emb_dim=config.d_model)
        self.summary = Summary(emb_dim=config.d_model)
        self.attn1 = nn.Linear(config.d_model, 5)
        self.attn2 = nn.Linear(5, config.d_model)  # Selected knowledge 개수만
        self.max_position = config.max_position_embeddings
        self.init_weights()

    def forward(
            self,
            input_ids=None,
            input_eos=None,
            only_dial_input_ids=None,
            decoder_input_ids=None,
            persona_input_ids=None,
            knowledge_input_ids=None,
            persona_can_idx=None,
            persona_grounding=None,
            knowledge_can_idx=None,
            knowledge_grounding=None,
            tot_knowledge=None,
            tot_knowledge_eos=None,
            training=None,
            lm_labels=None,
            mc_token_ids=None):

        machine = 50265
        human = 50266
        persona = 50267
        knowledge = 50268
        padding = 1
        bos = 0
        eos = 2
        num_chosen_paragraph = 5
        device = input_ids.get_device()

        persona_tensor = torch.tensor([persona]).cuda(device)
        knowledge_tensor = torch.tensor([knowledge]).cuda(device)
        bos_tensor = torch.tensor([bos]).cuda(device)
        eos_tensor = torch.tensor([eos]).cuda(device)

        outputs = tuple()
        dynamic_lm_logits = None
        persona_logits = None
        knowledge_logits = None

        # TODO : To find out - what is input_eos? Probably end of sentence for the inputs
        if input_eos is not None:
            lm_hidden_states = self.model(input_ids=input_ids)['last_hidden_state']
            batch, seq_len, embdim = lm_hidden_states.size()
            lm_hidden_states_eos_list = []
            for i in range(batch):
                lm_hidden_states_batch = lm_hidden_states[i]
                lm_eos_batch = input_eos[i]
                lm_hidden_states_eos = torch.index_select(lm_hidden_states_batch, -2, lm_eos_batch)
                lm_hidden_states_eos_list.append(lm_hidden_states_eos)
            lm_eos_rep = torch.stack(lm_hidden_states_eos_list)

            tot_knowledge_hidden_states = self.model(input_ids=tot_knowledge.view(batch*num_chosen_paragraph, -1))['last_hidden_state'].view(batch, num_chosen_paragraph, -1, embdim)
            tot_knowledge_eos_list = []
            for i in range(batch):
                tot_knowledge_hidden_states_batch = tot_knowledge_hidden_states[i]
                tot_knowledge_eos_batch = tot_knowledge_eos[i]
                tot_knowledge_eos_list_batch = []
                # TODO : This is hard coded (5), where does this come from?
                for j in range(5):
                    tot_knowledge_eos_token = torch.index_select(tot_knowledge_hidden_states_batch[j], -2, tot_knowledge_eos_batch[j])
                    tot_knowledge_eos_list_batch.append(tot_knowledge_eos_token.squeeze())
                tot_knowledge_eos_batch_rep = torch.stack(tot_knowledge_eos_list_batch)
                tot_knowledge_eos_list.append(tot_knowledge_eos_batch_rep)
            tot_knowledge_eos_final = torch.stack(tot_knowledge_eos_list)
            knowledge_inctxt_attn = self.attn1(tot_knowledge_eos_final)
            knowledge_inctxt_eos_rep = self.attn2(knowledge_inctxt_attn)
            inctxt_states = torch.cat((lm_eos_rep, knowledge_inctxt_eos_rep), dim=1).type_as(input_ids)

            sigmoid = Sigmoid()

            #persona candidates
            num_persona_can = 5
            if persona_input_ids is not None:
                persona_emb = self.model(input_ids=persona_input_ids.view(batch*num_persona_can,-1))['last_hidden_state'].view(batch, num_persona_can, -1, embdim)
                if persona_can_idx is not None:
                    persona_list = []
                    for batch_i in range(batch):
                        inctxt_eos_batch = inctxt_states[batch_i]
                        persona_emb_batch = persona_emb[batch_i]
                        persona_can_idx_batch = persona_can_idx[batch_i]
                        persona_batch_list = []
                        for i in range(num_persona_can):
                            persona_selected = torch.index_select(persona_emb_batch[i], 0, persona_can_idx_batch[i])
                            final_rep_persona = torch.cat([inctxt_eos_batch.type_as(lm_eos_rep), persona_selected.type_as(lm_eos_rep)], dim=0)
                            persona_batch_list.append(final_rep_persona)
                        persona_batch_list = torch.stack(persona_batch_list)
                        persona_list.append(persona_batch_list)
                    persona_rep = torch.stack(persona_list).view(batch*num_persona_can, -1)
                    persona_logits = self.concat_summary(persona_rep).view(batch, -1)
                    outputs = (persona_logits, )

                    persona_pred_sigmoid = sigmoid(persona_logits)
                    persona_pred_sigmoid = (persona_pred_sigmoid > 0.5).float()
                    all_persona_pred = []
                    selected_persona_idx = list()
                    for batch_idx, persona_batch in enumerate(torch.eq(persona_pred_sigmoid, 1)):
                        batch_list_idx = list()
                        batch_list = list()
                        for i, can in enumerate(persona_batch):
                            if can == True:
                                batch_list_idx.append(can)
                                persona_selected_now = persona_input_ids[batch_idx][i]
                                mask_persona = torch.ne(persona_selected_now, padding)
                                persona_selected_now = torch.masked_select(persona_selected_now, mask_persona)
                                batch_list.append(persona_selected_now[:-2])
                        all_persona_pred.append(batch_list)
                        selected_persona_idx.append(batch_list_idx)

            #knowledge candidates
            num_knowledge_can = 10
            if knowledge_input_ids is not None:
                knowledge_emb = self.model(input_ids=knowledge_input_ids.view(batch*num_knowledge_can, -1))['last_hidden_state'].view(batch, num_knowledge_can, -1, embdim)
                if knowledge_can_idx is not None:
                    knowledge_list = []
                    for batch_i in range(batch):
                        inctxt_eos_batch = inctxt_states[batch_i]
                        knowledge_emb_batch = knowledge_emb[batch_i]
                        knowledge_can_idx_batch = knowledge_can_idx[batch_i]
                        knowledge_batch_list = []
                        for i in range(num_knowledge_can):
                            knowledge_selected = torch.index_select(knowledge_emb_batch[i], 0, knowledge_can_idx_batch[i])
                            final_rep_knowledge = torch.cat([inctxt_eos_batch.type_as(lm_eos_rep), knowledge_selected.type_as(lm_eos_rep)], dim=0)
                            knowledge_batch_list.append(final_rep_knowledge)
                        knowledge_batch_list = torch.stack(knowledge_batch_list)
                        knowledge_list.append(knowledge_batch_list)
                        # TODO : MODIFY
                        # import pdb; pdb.set_trace()

                    knowledge_rep = torch.stack(knowledge_list).view(batch*num_knowledge_can, -1)
                    knowledge_logits = self.concat_summary(knowledge_rep).view(batch, -1)
                    outputs = (knowledge_logits,) + outputs
                    softmax = Softmax(dim=-1)
                    knowledge_softmax = softmax(knowledge_logits)
                    _, k_index_1 = torch.topk(knowledge_softmax, k=1, dim=-1)
                    all_knowledge_pred = []
                    for batch_i in range(batch):
                        knowledge_pred_idx = k_index_1[batch_i]
                        knowledge_pred = knowledge_input_ids[batch_i][knowledge_pred_idx]
                        mask_knowledge = torch.ne(knowledge_pred, padding)
                        knowledge_pred = torch.masked_select(knowledge_pred, mask_knowledge)
                        knowledge_pred = knowledge_pred[1:-2]
                        all_knowledge_pred.append(knowledge_pred) #delete bos, knowledge_st, eos


            final_input_list = []
            for batch_i in range(batch):
                only_dial_input_ids_batch = only_dial_input_ids[batch_i]
                mask_only_dial_input_ids_batch = torch.ne(only_dial_input_ids_batch, padding)
                only_dial_input_ids_batch = torch.masked_select(only_dial_input_ids_batch, mask_only_dial_input_ids_batch)
                if len(all_persona_pred[batch_i])>0:
                    concat_persona = torch.cat(all_persona_pred[batch_i], dim=-1)
                    new_persona = torch.cat([persona_tensor, concat_persona], dim=-1)
                else:
                    new_persona = None

                new_knowledge = torch.cat([knowledge_tensor, all_knowledge_pred[batch_i]], dim=-1)

                if new_persona is not None:
                    new_input = torch.cat([bos_tensor, new_knowledge, new_persona, only_dial_input_ids_batch, eos_tensor], dim=-1)
                else:
                    new_input = torch.cat([bos_tensor, new_knowledge, only_dial_input_ids_batch, eos_tensor], dim=-1)

                new_input_size = new_input.size()[0]


                if new_input_size < int(self.max_position) :
                    padding_size = int(self.max_position) - new_input_size
                    add_padding = torch.tensor([padding]*padding_size).cuda(device)
                    final_input = torch.cat([new_input, add_padding], dim=-1)
                final_input_list.append(final_input)
            input_ids = torch.stack(final_input_list)
        # input : <human> Hello <machine> What can I do for you? <human> ... <persona> adlkasjdlkaj <knowledge> asldkjasldkj
        dynamic_lm_hidden_states = self.model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)['last_hidden_state']

        if dynamic_lm_hidden_states is not None:
            dynamic_lm_logits = self.lm_head(dynamic_lm_hidden_states)
            outputs = (dynamic_lm_logits,) + outputs

        if persona_grounding is not None:
            loss_fct = BCEWithLogitsLoss()
            persona_loss = loss_fct(persona_logits.view(batch, -1), persona_grounding.type_as(persona_logits))
            outputs = (persona_loss,) + outputs

        if knowledge_grounding is not None:
            loss_fct = CrossEntropyLoss()
            knowledge_loss = loss_fct(knowledge_logits.view(batch, -1), knowledge_grounding)
            outputs = (knowledge_loss,) + outputs

        if training is True:
            shift_logits = dynamic_lm_logits[..., :-1, :].contiguous() # <sos> ..... 
            shift_labels = lm_labels[..., 1:].contiguous() # ... <eos>
            loss_fct = CrossEntropyLoss(ignore_index=-100) # padding, -> loss
            lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            outputs = (lm_loss,) + outputs


        return 

In [ ]:
import os
import math
import logging
from pprint import pformat
from argparse import ArgumentParser
import torch
from torch.nn import Sigmoid, Softmax
from torch.nn.parallel import DistributedDataParallel
from ignite.engine import Engine, Events
from ignite.handlers import ModelCheckpoint
from ignite.metrics import Loss, MetricsLambda, RunningAverage, Precision, CharFbeta, Recall, Accuracy
from ignite.metrics import Bleu, RougeL, RougeN
from ignite.contrib.handlers import ProgressBar, PiecewiseLinear
from ignite.contrib.handlers.tensorboard_logger import TensorboardLogger, OutputHandler, OptimizerParamsHandler
from transformers import (AdamW, WEIGHTS_NAME, CONFIG_NAME)
from utils_focus import make_focus_logdir
from data_utils import get_data_loaders, add_special_tokens_

logger = logging.getLogger(__file__)


def average_distributed_scalar(scalar, args):
    """ Average a scalar over the nodes if we are in distributed training. We use this for distributed evaluation. """
    if args.local_rank == -1:
        return scalar
    scalar_t = torch.tensor(scalar, dtype=torch.float, device=args.device) / torch.distributed.get_world_size()
    torch.distributed.all_reduce(scalar_t, op=torch.distributed.ReduceOp.SUM)
    return scalar_t.item()


def train():
    parser = ArgumentParser()
    parser.add_argument("--model_name", type=str, default="",
                        help="{GPT2, BART, transformer-decoder, transformer-encdec}")
    parser.add_argument("--gpt2_model_path", type=str, default="gpt2",
                        help="pre-trained model path for decoder only models")  # gpt2-medium
    parser.add_argument("--bart_model_path", type=str, default="facebook/bart-base",
                        help="pre-trained model path for encoder-decoder models")  # facebook/bart-large
    parser.add_argument("--train_dataset_path", type=str, default="data/train_focus.json",
                        help="Path or url of the dataset.")
    parser.add_argument("--train_dataset_cache", type=str, default='data/focus_cache.tar.gz',
                        help="Path or url of the dataset cache")
    parser.add_argument("--dev_dataset_path", type=str, default="data/valid_focus.json",
                        help="Path or url of the dataset.")
    parser.add_argument("--dev_dataset_cache", type=str, default='data/focus_cache.tar.gz',
                        help="Path or url of the dataset cache")
    parser.add_argument("--ps_coef", type=float, default=1.0, help="Coefficient for persona loss")
    parser.add_argument("--kn_coef", type=float, default=1.0, help="Coefficient for knowledge loss")
    parser.add_argument("--lm_coef", type=float, default=10.0, help="Coefficient for LM loss")
    parser.add_argument("--max_history", type=int, default=1, help="Number of previous exchanges to keep in history")
    parser.add_argument("--train_batch_size", type=int, default=4, help="Batch size for training")
    parser.add_argument("--valid_batch_size", type=int, default=1, help="Batch size for validation")
    parser.add_argument("--gradient_accumulation_steps", type=int, default=16,
                        help="Accumulate gradients on several steps")
    parser.add_argument("--lr", type=float, default=6.25e-5, help="Learning rate")
    parser.add_argument("--max_norm", type=float, default=1.0, help="Clipping gradient norm")
    parser.add_argument("--n_epochs", type=int, default=1, help="Number of training epochs")
    parser.add_argument("--eval_before_start", action='store_true',
                        help="If true start with a first evaluation before training")
    parser.add_argument("--inference", action='store_true', help="If true, inference with gold knowledge")
    parser.add_argument("--test_infer", action='store_true', help="If true, test inference")
    parser.add_argument("--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu",
                        help="Device (cuda or cpu)")
    parser.add_argument("--fp16", type=str, default="",
                        help="Set to O0, O1, O2 or O3 for fp16 training (see apex documentation)")
    parser.add_argument("--local_rank", type=int, default=-1,
                        help="Local rank for distributed training (-1: not distributed)")
    parser.add_argument("--gpu_start_num", type=int, default=1, help="Start number of GPU")
    parser.add_argument("--flag", type=str, default="", help="Assign the name of the folder")
    parser.add_argument("--seed", type=int, default=19950604)
    parser.add_argument("--random_knowledge", action='store_true',
                        help="If true, the model choose the knowledge randomly")
    parser.add_argument("--incontext", action='store_true', help="If true, it will use incontext structure")
    parser.add_argument("--append_k_triples", action='store_true', help="If true, inference with triples instead of knowledge")
    parser.add_argument("--replace_k_triples", action='store_true', help="If true, inference with triples instead of knowledge")
    args = parser.parse_args()
    torch.manual_seed(args.seed)

    logging.basicConfig(level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN)
    logger.info("Arguments: %s", pformat(args))

    args.distributed = (args.local_rank != -1)
    if args.distributed:
        local_rank = args.local_rank + args.gpu_start_num
        print("args local rank: ", args.local_rank, " local rank: ", local_rank)
        torch.cuda.set_device(local_rank)
        args.device = torch.device("cuda", local_rank)
        torch.distributed.init_process_group(backend='nccl', init_method='env://')

    logger.info("Prepare tokenizer, pretrained model and optimizer.")

    if args.model_name == 'GPT2':
        from transformers import GPT2Tokenizer
        from classification_modules import GPT2PK_ctxt as gpt2model
        tokenizer = GPT2Tokenizer.from_pretrained(args.gpt2_model_path)
        model = gpt2model.from_pretrained(args.gpt2_model_path)
        model.to(args.device)
        model.eval()
        if args.gpt2_model_path == 'gpt2' or 'gpt2-medium':
            add_special_tokens_(model, tokenizer)

    elif args.model_name == 'BART':
        from transformers import BartTokenizer
        from classification_modules import BARTPK_ctxt as bartmodel
        tokenizer = BartTokenizer.from_pretrained(args.bart_model_path)
        model = bartmodel.from_pretrained(args.bart_model_path)
        model.to(args.device)
        model.eval()
        if args.bart_model_path == "facebook/bart-base" or "facebook/bart-large":
            add_special_tokens_(model, tokenizer)

    elif args.model_name == 'transformer-decoder':
        from transformers import GPT2Tokenizer, GPT2Config
        from classification_modules import GPT2PK_ctxt as gpt2model
        tokenizer = GPT2Tokenizer.from_pretrained(args.gpt2_model_path)
        model_config = GPT2Config.from_pretrained(args.gpt2_model_path)
        model = gpt2model(model_config)
        model.to(args.device)
        if args.gpt2_model_path == 'gpt2' or 'gpt2-medium':
            add_special_tokens_(model, tokenizer)

    elif args.model_name == 'transformer-encdec':
        from transformers import BartTokenizer, BartConfig
        from classification_modules import BARTPK_ctxt as bartmodel
        tokenizer = BartTokenizer.from_pretrained(args.bart_model_path)
        model_config = BartConfig.from_pretrained(args.bart_model_path)
        model = bartmodel(model_config)
        model.to(args.device)
        if args.bart_model_path == "facebook/bart-base" or "facebook/bart-large":
            add_special_tokens_(model, tokenizer)

    else:
        raise NotImplementedError

    optimizer = AdamW(model.parameters(), lr=args.lr, correct_bias=True)

    if args.fp16:
        from apex import amp  # Apex is only required if we use fp16 training
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16)
    if args.distributed:
        model = DistributedDataParallel(model, device_ids=[local_rank], output_device=local_rank)

    logger.info("Prepare datasets")
    train_loader, val_loader, train_sampler, valid_sampler = get_data_loaders(args, tokenizer)

    # Training function and trainer
    def update(engine, batch):
        model.train()
        batch = tuple(input_tensor.to(args.device) for input_tensor in batch)
        if model.config.model_type == 'gpt2':
            input_ids, input_eos, lm_labels, token_type_ids, mc_token_ids, persona_candidates, persona_can_idx, persona_grounding, knowledge_candidates, \
            knowledge_can_idx, knowledge_grounding, tot_knowledge, tot_knowledge_token_ids, tot_knowledge_eos, reply, dialog, dialog_tti = batch
            output = model(
                input_ids=input_ids,
                input_eos=input_eos,
                token_type_ids=token_type_ids,
                only_dial_input_ids=dialog,
                only_dial_token_type_ids=dialog_tti,
                persona_input_ids=persona_candidates,
                knowledge_input_ids=knowledge_candidates,
                persona_can_idx=persona_can_idx,
                persona_grounding=persona_grounding,
                knowledge_can_idx=knowledge_can_idx,
                knowledge_grounding=knowledge_grounding,
                tot_knowledge=tot_knowledge,
                tot_knowledge_token_ids=tot_knowledge_token_ids,
                tot_knowledge_eos=tot_knowledge_eos,
                training=True,
                mc_token_ids=mc_token_ids
            )
        elif model.config.model_type == 'bart':
          # dialogue token ids, dialogue labels, persona token ids, persona labels, knowledge token ids, knowledge labels
            input_ids, input_eos, decoder_input_ids, lm_labels, token_type_ids, mc_token_ids, persona_candidates, persona_can_idx, persona_grounding, knowledge_candidates, \
            knowledge_can_idx, knowledge_grounding, tot_knowledge, tot_knowledge_eos, reply, dialog, filtered_triples, ner_cans, triple_plus_knowledge = batch
            # import pdb; pdb.set_trace()
            assert len(filtered_triples) == len(knowledge_candidates)
            output = model(
                input_ids=input_ids,
                input_eos=input_eos,
                only_dial_input_ids=dialog,
                decoder_input_ids=decoder_input_ids,
                persona_input_ids=persona_candidates,
                knowledge_input_ids=knowledge_candidates if not args.replace_k_triples else filtered_triples \
                    if not args.append_k_triples else triple_plus_knowledge,
                persona_can_idx=persona_can_idx,
                persona_grounding=persona_grounding,
                knowledge_can_idx=knowledge_can_idx,
                knowledge_grounding=knowledge_grounding,
                tot_knowledge=tot_knowledge,
                tot_knowledge_eos=tot_knowledge_eos,
                lm_labels=lm_labels,
                training=True,
                mc_token_ids=mc_token_ids
            )
        else:
            raise NotImplementedError
        lm_loss, knowledge_loss, persona_loss = output[0], output[1], output[2]
        loss = (lm_loss * args.lm_coef + knowledge_loss * args.kn_coef + persona_loss * args.ps_coef) / args.gradient_accumulation_steps
        if args.fp16:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_norm)
        else:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_norm)
        if engine.state.iteration % args.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
        return (lm_loss.item(), knowledge_loss.item(), persona_loss.item())

    trainer = Engine(update)

    # Evaluation function and evaluator (evaluator output is the input of the metrics)
    def inference(engine, batch):
        model.eval()
        with torch.no_grad():
            batch = tuple(input_tensor.to(args.device) for input_tensor in batch)
            if model.config.model_type == 'gpt2':
                input_ids, input_eos, lm_labels, token_type_ids, mc_token_ids, persona_candidates, persona_can_idx, persona_grounding, knowledge_candidates, \
                knowledge_can_idx, knowledge_grounding, tot_knowledge, tot_knowledge_token_ids, tot_knowledge_eos, reply, dialog, dialog_tti = batch

                output = model(
                    input_ids=input_ids,
                    input_eos=input_eos,
                    token_type_ids=token_type_ids,
                    only_dial_input_ids=dialog,
                    only_dial_token_type_ids=dialog_tti,
                    persona_input_ids=persona_candidates,
                    knowledge_input_ids=knowledge_candidates,
                    persona_can_idx=persona_can_idx,
                    knowledge_can_idx=knowledge_can_idx,
                    tot_knowledge=tot_knowledge,
                    tot_knowledge_token_ids=tot_knowledge_token_ids,
                    tot_knowledge_eos=tot_knowledge_eos,
                    training=False,
                    mc_token_ids=mc_token_ids
                )
                lm_labels, lm_logits, knowledge_logits, persona_logits = output[0], output[1], output[2], output[3]


            elif model.config.model_type == 'bart':
                input_ids, input_eos, decoder_input_ids, lm_labels, token_type_ids, mc_token_ids, persona_candidates, persona_can_idx, persona_grounding, knowledge_candidates, \
                knowledge_can_idx, knowledge_grounding, tot_knowledge, tot_knowledge_eos, reply, dialog, filtered_triples, ner_cans, triple_plus_knowledge = batch
                assert len(filtered_triples) == len(knowledge_candidates)
                output = model(
                    input_ids=input_ids,
                    input_eos=input_eos,
                    only_dial_input_ids=dialog,
                    decoder_input_ids=decoder_input_ids,
                    persona_input_ids=persona_candidates,
                    knowledge_input_ids=knowledge_candidates if not args.replace_k_triples else filtered_triples \
                        if not args.append_k_triples else triple_plus_knowledge,
                    persona_can_idx=persona_can_idx,
                    knowledge_can_idx=knowledge_can_idx,
                    tot_knowledge=tot_knowledge,
                    tot_knowledge_eos=tot_knowledge_eos,
                    training=False,
                    mc_token_ids=mc_token_ids
                )
                lm_logits, knowledge_logits, persona_logits = output[0], output[1], output[2]


            else:
                raise NotImplementedError

            lm_logits_flat_shifted = lm_logits[:, :-1, :].contiguous().view(-1, lm_logits.size(-1))
            lm_labels_flat_shifted = lm_labels[:, 1:].contiguous().view(-1)

            persona_logits = persona_logits.squeeze()
            persona_grounding = persona_grounding.type_as(persona_logits).squeeze()

            sigmoid = Sigmoid()
            persona_pred_sigmoid = sigmoid(persona_logits)
            persona_pred_sigmoid = (persona_pred_sigmoid > 0.5).float()

            softmax = Softmax(dim=-1)
            knowledge_pred = softmax(knowledge_logits)
            _, k_index_1 = torch.topk(knowledge_pred, k=1, dim=-1)
            _, k_index_5 = torch.topk(knowledge_pred, k=5, dim=-1)
            k_index_1, k_index_5 = k_index_1.squeeze(0), k_index_5.squeeze(0)
            k_index_1_cvtd = torch.tensor([1 if num in k_index_1 else 0 for num in range(10)], device=args.device)
            k_label_cvtd = torch.tensor([1 if num in knowledge_grounding else 0 for num in range(10)],
                                        device=args.device)

            lm_pred = softmax(lm_logits_flat_shifted)
            lm_val, lm_idx = torch.topk(lm_pred, k=1, dim=-1)
            lm_idx = lm_idx.squeeze(-1)

            mask = (lm_labels_flat_shifted != -100)
            lm_labels_only = [lm_labels_flat_shifted[mask].tolist()]
            lm_idx_only = lm_idx[mask].tolist()

            return (lm_logits_flat_shifted, knowledge_logits, persona_logits, persona_pred_sigmoid, k_index_1_cvtd, knowledge_pred, lm_idx_only), \
                   (lm_labels_flat_shifted, knowledge_grounding, persona_grounding.type_as(persona_logits), k_label_cvtd, lm_labels_only)


    evaluator = Engine(inference)

    # Attach evaluation to trainer: we evaluate when we start the training and at the end of each epoch
    trainer.add_event_handler(Events.EPOCH_COMPLETED, lambda _: evaluator.run(val_loader))
    if args.n_epochs < 1:
        trainer.add_event_handler(Events.COMPLETED, lambda _: evaluator.run(val_loader))
    if args.eval_before_start:
        trainer.add_event_handler(Events.STARTED, lambda _: evaluator.run(val_loader))

    # Make sure distributed data samplers split the dataset nicely between the distributed processes
    if args.distributed:
        trainer.add_event_handler(Events.EPOCH_STARTED, lambda engine: train_sampler.set_epoch(engine.state.epoch))
        evaluator.add_event_handler(Events.EPOCH_STARTED, lambda engine: valid_sampler.set_epoch(engine.state.epoch))

    # Linearly decrease the learning rate from lr to zero
    scheduler = PiecewiseLinear(optimizer, "lr", [(0, args.lr), (args.n_epochs * len(train_loader), 0.0)])
    trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)

    # Prepare metrics - note how we compute distributed metrics
    RunningAverage(output_transform=lambda x: x[0]).attach(trainer, "lm_loss")
    RunningAverage(output_transform=lambda x: x[1]).attach(trainer, "knowledge_loss")
    RunningAverage(output_transform=lambda x: x[2]).attach(trainer, "persona_loss")

    metrics = {
        "lm_loss": Loss(torch.nn.CrossEntropyLoss(ignore_index=-100), output_transform=lambda x: (x[0][0], x[1][0])),
        "knowledge_loss": Loss(torch.nn.CrossEntropyLoss(), output_transform=lambda x: (x[0][1], x[1][1])),
        "persona_loss": Loss(torch.nn.BCEWithLogitsLoss(), output_transform=lambda x: (x[0][2], x[1][2])),
        "Knowledge_acc": Accuracy(output_transform=lambda x: (x[0][5], x[1][3])),
        "Persona_acc":Accuracy(output_transform=lambda x: (x[0][3], x[1][2]))}

    metrics.update({"average_lm_loss": MetricsLambda(average_distributed_scalar, metrics["lm_loss"], args),
                    "average_knowledge_loss": MetricsLambda(average_distributed_scalar, metrics["knowledge_loss"],args),
                    "average_persona_loss": MetricsLambda(average_distributed_scalar, metrics["persona_loss"], args),
                    "average_Knowledge_acc": MetricsLambda(average_distributed_scalar, metrics["Knowledge_acc"], args),
                    "average_Persona_acc": MetricsLambda(average_distributed_scalar, metrics["Persona_acc"], args)
                    })

    metrics["average_ppl"] = MetricsLambda(math.exp, metrics["average_lm_loss"])

    for name, metric in metrics.items():
        metric.attach(evaluator, name)

    # On the main process: add progress bar, tensorboard, checkpoints and save model, configuration and tokenizer before we start to train
    if args.local_rank in [-1, 0]:
        pbar = ProgressBar(persist=True)
        pbar.attach(trainer, metric_names=["lm_loss"])
        evaluator.add_event_handler(Events.COMPLETED,
                                    lambda _: pbar.log_message("Validation: %s" % pformat(evaluator.state.metrics)))

        dir_name = str(os.path.basename(__file__))[:-3] + "_" + args.model_name + "_" + args.flag
        log_dir = make_focus_logdir(dir_name)
        tb_logger = TensorboardLogger(log_dir)

        tb_logger.attach(trainer, log_handler=OutputHandler(tag="training", metric_names=["lm_loss", "knowledge_loss", "persona_loss",
                                                                          "knowledge_accuracy", "persona_accuracy", "f1_score"]),
                         event_name=Events.ITERATION_COMPLETED)
        tb_logger.attach(trainer, log_handler=OptimizerParamsHandler(optimizer), event_name=Events.ITERATION_STARTED)
        tb_logger.attach(evaluator, log_handler=OutputHandler(tag="validation", metric_names=list(metrics.keys())),
                         event_name=Events.ITERATION_STARTED)
        tb_logger.attach(evaluator, log_handler=OutputHandler(tag="validation", metric_names=list(metrics.keys())),
                         event_name=Events.EPOCH_STARTED)
        tb_logger.attach(evaluator, log_handler=OutputHandler(tag="validation", metric_names=list(metrics.keys())),
                         event_name=Events.ITERATION_COMPLETED(every=5000))
        checkpoint_handler = ModelCheckpoint(log_dir, 'checkpoint', n_saved=3)
        trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {
            'mymodel': getattr(model, 'module', model)})  # "getattr" takes care of distributed encapsulation

        torch.save(args, log_dir + '/model_training_args.bin')
        getattr(model, 'module', model).config.to_json_file(os.path.join(log_dir, CONFIG_NAME))
        tokenizer.save_pretrained(log_dir)

    # Run the training
    trainer.run(train_loader, max_epochs=args.n_epochs)

    # On the main process: close tensorboard logger and rename the last checkpoint (for easy re-loading with OpenAIGPTModel.from_pretrained method)
    if args.local_rank in [-1, 0] and args.n_epochs > 0:
        os.rename(os.path.join(log_dir, checkpoint_handler._saved[-1][1]), os.path.join(log_dir, WEIGHTS_NAME))
        tb_logger.close()


if __name__ == "__main__":
    train()

In [ ]:
#(c) 2021 NCSOFT Corporation & Korea University. All rights reserved.
import logging
import random
from argparse import ArgumentParser
from pprint import pformat
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.nn import Sigmoid, Softmax, CrossEntropyLoss
from data_utils import get_testdata_loaders, add_special_tokens_
from inference_test import sample_sequence
from ignite.metrics import Accuracy
from datasets import load_metric
from torchmetrics import CHRFScore
from rouge_score import rouge_scorer
logger = logging.getLogger(__file__)
from setproctitle import setproctitle
setproctitle('Yoonna')

SPECIAL_TOKENS = ["<machine>", "<human>", "<persona>", "<knowledge>"]

# TODO : Augment with https://en.wikipedia.org/wiki/Most_common_words_in_English
# Rough version allowing knowledge duplicates

def run():
    parser = ArgumentParser()
    parser.add_argument("--test_dataset_path", type=str, default="data/new_valid_json_v3.json", help="Path or url of the dataset. If empty download from S3.") #data
    parser.add_argument("--test_dataset_cache", type=str, default='data/new_focus_cache_v5_ner_wo_rel.tar.gz', help="Path or url of the dataset cache")
    parser.add_argument("--model_name", type=str, default="", help="{GPT2, BART, transformer-decoder, transformer-encdec}")
    parser.add_argument("--model_checkpoint", type=str, default="", help="Path, url or short name of the model")
    parser.add_argument("--max_history", type=int, default=1, help="Number of previous utterances to keep in history")
    parser.add_argument("--test_batch_size", type=int, default=1, help="Batch size for testing")
    parser.add_argument("--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu", help="Device (cuda or cpu)")
    parser.add_argument("--no_sample", action='store_true', help="Set to use greedy decoding instead of sampling")
    parser.add_argument("--max_length", type=int, default=20, help="Maximum length of the output utterances")
    parser.add_argument("--min_length", type=int, default=1, help="Minimum length of the output utterances")
    parser.add_argument("--select_max_len", action='store_true', help="Select maximum length beam")
    parser.add_argument("--inference", action='store_true', help="If true, inference with gold knowledge")
    parser.add_argument("--seed", type=int, default=19950604, help="Seed")
    parser.add_argument("--temperature", type=int, default=0.7, help="Sampling softmax temperature")
    parser.add_argument("--top_k", type=int, default=0, help="Filter top-k tokens before sampling (<=0: no filtering)")
    parser.add_argument("--top_p", type=float, default=0.9, help="Nucleus filtering (top-p) before sampling (<=0.0: no filtering)")
    parser.add_argument("--local_rank", type=int, default=-1, help="Local rank for distributed training (-1: not distributed)")
    parser.add_argument("--literal", type=float, default=1.0, help="Constraining based on literal")
    parser.add_argument("--graph", type=float, default=1.0, help="Constraining based on graph")
    parser.add_argument("--ner", type=float, default=1.0, help="Constraining based on NER")
    parser.add_argument("--eos_std_adj", type=float, default=0.0, help="Equation to adjust EOS token probability")
    parser.add_argument("--markov_graph", type=float, default=1.0, help="Constraining based on markov characteristics of graph")
    parser.add_argument("--markov_graph_v2", type=float, default=1.0, help="Constraining based on graph subject utilizing all relation / objects")
    parser.add_argument("--constrain", action='store_true', help="New constraining mechanism")
    parser.add_argument("--gen_eval", action='store_true', help="Generation Auto Evaluation")
    parser.add_argument("--without_rel", action="store_true", help="Whether to remove relation from graph data")
    parser.add_argument("--submit_path", type=str, default="", help="Line-by-line output for submission")
    parser.add_argument("--beam_size", type=int, default=1, help="Beam size (currently only performs basic override)")
    parser.add_argument("--beam_length_alpha", type=float, default=0.0, help="Beam alpha discount")
    parser.add_argument("--no_eval", action='store_true', help="No evaluation mode.")
    parser.add_argument("--line_proc", action='store_true', help="If true, the file is processed into lines.")
    args = parser.parse_args()

    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__file__)
    logger.info(pformat(args))
    args.distributed = (args.local_rank != -1)

    if args.seed != 0:
    	random.seed(args.seed)
        torch.random.manual_seed(args.seed)
        torch.cuda.manual_seed(args.seed)

    logger.info("Get model and tokenizer")
    logger.info(f"Submit to : {args.submit_path}")

    if args.model_name == 'GPT2':
        from transformers import GPT2Tokenizer
        from classification_modules import GPT2PK_ctxt
        tokenizer = GPT2Tokenizer.from_pretrained(args.model_checkpoint)
        model = GPT2PK_ctxt.from_pretrained(args.model_checkpoint)
        model.to(args.device)
        add_special_tokens_(model, tokenizer)

    elif args.model_name == 'BART':
        from transformers import BartTokenizer
        from classification_modules import BARTPK_ctxt
        tokenizer = BartTokenizer.from_pretrained(args.model_checkpoint)
        model = BARTPK_ctxt.from_pretrained(args.model_checkpoint)
        logger.info("Model loaded")

        model.to(args.device)
        logger.info("To Device")
        add_special_tokens_(model, tokenizer)

    elif args.model_name == 'transformer-decoder':
        from transformers import GPT2Tokenizer
        from classification_modules import GPT2PK_ctxt
        tokenizer = GPT2Tokenizer.from_pretrained(args.model_checkpoint)
        model = GPT2PK_ctxt.from_pretrained(args.model_checkpoint)
        model.to(args.device)
        add_special_tokens_(model, tokenizer)

    elif args.model_name == 'transformer-encdec':
        from transformers import BartTokenizer
        from classification_modules import BARTPK_ctxt
        tokenizer = BartTokenizer.from_pretrained(args.model_checkpoint)
        model = BARTPK_ctxt.from_pretrained(args.model_checkpoint)
        model.to(args.device)
        add_special_tokens_(model, tokenizer)

    else:
        raise NotImplementedError


    #dataset = get_dataset_only_train(tokenizer, args.dataset_path, args.dataset_cache)
    logger.info("Prepare datasets")
    test_loader, test_sampler = get_testdata_loaders(args, tokenizer, generation=True)

    with torch.no_grad():
        r1 = 0
        r2 = 0
        rl = 0
        bleu = 0
        bertscore_precision = 0
        bertscore_recall = 0
        bertscore_f1 = 0
        # bleurt = 0
        mauve = 0
        chrf1 = 0
        rouge_metric = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        bleu_metric = load_metric("sacrebleu")
        bertscore_metric = load_metric("bertscore")
        # bleurt_metric = load_metric("bleurt")
        # mauve_metric = load_metric("mauve")
        chrf_metric = CHRFScore()

        pg = Accuracy()
        kg = Accuracy()

        pass_num = 0
        test_pass_list = []

        import collections
        submit_output = collections.defaultdict(list)

        # import pdb; pdb.set_trace()
        for test_data_index, test_data in enumerate(tqdm(test_loader)):
            # import pdb; pdb.set_trace()
            loss_fct = CrossEntropyLoss(ignore_index=-100)
            # TODO : Update can_idx for knowledge and persona when testing
            if model.config.model_type == 'gpt2':
                input_ids, input_eos, lm_labels, token_type_ids, mc_token_ids, persona_candidates, persona_can_idx, persona_grounding, knowledge_candidates, \
                knowledge_can_idx, knowledge_grounding, tot_knowledge, tot_knowledge_token_ids, tot_knowledge_eos, reply, dialog, dialog_tti = test_data

            elif model.config.model_type == 'bart':
                # import pdb; pdb.set_trace()
                input_ids, input_eos, decoder_input_ids, lm_labels, token_type_ids, mc_token_ids, persona_candidates, persona_can_idx, persona_grounding, knowledge_candidates, \
                knowledge_can_idx, knowledge_grounding, tot_knowledge, tot_knowledge_eos, reply, dialog, dialog_ID, filtered_triples, filtered_triples_raw,\
                ner_cans = test_data
                # import pdb; pdb.set_trace()
            else:
                raise NotImplementedError

            reply = reply[0]
            mask = (reply != tokenizer.pad_token_id)
            reply = reply[mask]
            # import pdb; pdb.set_trace()

            #print('reply.tolist() ', reply.tolist())
            if len(reply.tolist()) == 0:
                pass_num += 1
                test_pass_list.append(test_data_index)
                continue

            if model.config.model_type == 'gpt2':
                output = model(
                    input_ids=input_ids,
                    input_eos=input_eos,
                    token_type_ids=token_type_ids,
                    only_dial_input_ids=dialog,
                    only_dial_token_type_ids=dialog_tti,
                    persona_input_ids=persona_candidates,
                    knowledge_input_ids=knowledge_candidates,
                    persona_can_idx=persona_can_idx,
                    knowledge_can_idx=knowledge_can_idx,
                    tot_knowledge=tot_knowledge,
                    tot_knowledge_token_ids=tot_knowledge_token_ids,
                    tot_knowledge_eos=tot_knowledge_eos,
                    training=False,
                    mc_token_ids=mc_token_ids
                )
                lm_labels, lm_logits, knowledge_logits, persona_logits = output[0], output[1], output[2], output[3]

                machine, human, persona, knowledge, padding, bos = 50257, 50258, 50259, 50260, 50261, 50256
                device = input_ids.get_device()

                machine_tensor = torch.tensor([machine]).cuda(device)
                persona_tensor = torch.tensor([persona]).cuda(device)
                knowledge_tensor = torch.tensor([knowledge]).cuda(device)
                bos_tensor = torch.tensor([bos]).cuda(device)


                sigmoid = Sigmoid()
                persona_pred_sigmoid = sigmoid(persona_logits)
                persona_pred_sigmoid = (persona_pred_sigmoid > 0.5).float()
                all_persona_pred = []
                selected_persona_idx = list()
                for batch_idx, persona_batch in enumerate(torch.eq(persona_pred_sigmoid, 1)):
                    batch_list_idx = list()
                    batch_list = list()
                    for i, can in enumerate(persona_batch):
                        if can == True:
                            batch_list_idx.append(can)
                            persona_selected_now = persona_candidates[batch_idx][i]
                            mask_persona = torch.ne(persona_selected_now, padding)
                            persona_selected_now = torch.masked_select(persona_selected_now, mask_persona)
                            batch_list.append(persona_selected_now[:-2])
                    all_persona_pred.append(batch_list)
                    selected_persona_idx.append(batch_list_idx)
                p_index_cvtd = persona_pred_sigmoid


                softmax = Softmax(dim=-1)
                knowledge_softmax = softmax(knowledge_logits)
                _, k_index_1 = torch.topk(knowledge_softmax, k=1, dim=-1)
                all_knowledge_pred = []
                for batch_i in range(args.test_batch_size):
                    knowledge_pred_idx = k_index_1[batch_i]
                    knowledge_pred = knowledge_candidates[batch_i][knowledge_pred_idx]
                    mask_knowledge = torch.ne(knowledge_pred, padding)
                    knowledge_pred = torch.masked_select(knowledge_pred, mask_knowledge)
                    knowledge_pred = knowledge_pred[1:-2]
                    all_knowledge_pred.append(knowledge_pred) #delete bos, knowledge_st, eos


                k_index_1 = k_index_1.squeeze(0)
                k_index_cvtd = torch.tensor([1 if num in k_index_1 else 0 for num in range(10)], device=args.device)

                final_input_list = []
                final_input_tti_list = []
                for batch_i in range(args.test_batch_size):
                    only_dial_input_ids_batch = dialog[batch_i]
                    only_dial_token_type_ids_batch = dialog_tti[batch_i]
                    mask_only_dial_input_ids_batch = torch.ne(only_dial_input_ids_batch, padding)
                    mask_only_dial_tti_batch = torch.ne(only_dial_token_type_ids_batch, padding)
                    only_dial_input_ids_batch = torch.masked_select(only_dial_input_ids_batch, mask_only_dial_input_ids_batch)
                    only_dial_token_type_ids_batch = torch.masked_select(only_dial_token_type_ids_batch, mask_only_dial_tti_batch)

                    if len(all_persona_pred[batch_i]) > 0:
                        concat_persona = torch.cat(all_persona_pred[batch_i], dim=-1)
                        new_persona = torch.cat([persona_tensor, concat_persona], dim=-1)
                        new_persona_tti = torch.tensor([persona] * (new_persona.size()[0])).cuda(device)

                    else:
                        new_persona = None
                        new_persona_tti = None


                    new_knowledge = torch.cat([knowledge_tensor, all_knowledge_pred[batch_i]], dim=-1)
                    new_knowledge_tti = torch.tensor([knowledge] * (new_knowledge.size()[0])).cuda(device)

                    only_dial_input_ids_batch = only_dial_input_ids_batch[1:-1]
                    only_dial_token_type_ids_batch = only_dial_token_type_ids_batch[1:]
                    if new_persona is not None:
                        new_input = torch.cat([bos_tensor, new_knowledge, new_persona, only_dial_input_ids_batch, machine_tensor], dim=-1)
                        new_input_tti = torch.cat([knowledge_tensor, new_knowledge_tti, new_persona_tti, only_dial_token_type_ids_batch], dim=-1)
                    else:
                        new_input = torch.cat([bos_tensor, new_knowledge, only_dial_input_ids_batch, machine_tensor], dim=-1)
                        new_input_tti = torch.cat([knowledge_tensor, new_knowledge_tti, only_dial_token_type_ids_batch], dim=-1)

                    final_input_list.append(new_input)
                    final_input_tti_list.append(new_input_tti)
                final_input_tensor = torch.stack(final_input_list)
                final_input_tti_tensor = torch.stack(final_input_tti_list)

                out_ids = sample_sequence(final_input_tensor, token_type_ids=final_input_tti_tensor,
                                          decoder_input_ids=None, tokenizer=tokenizer, model=model, args=args, current_output=None)

            elif model.config.model_type == 'bart':
                # import pdb; pdb.set_trace()
                output = model(
                    input_ids=input_ids,
                    input_eos=input_eos,
                    only_dial_input_ids=dialog,
                    decoder_input_ids=decoder_input_ids,
                    persona_input_ids=persona_candidates,
                    knowledge_input_ids=knowledge_candidates,
                    persona_can_idx=persona_can_idx,
                    knowledge_can_idx=knowledge_can_idx,
                    tot_knowledge=tot_knowledge,
                    tot_knowledge_eos=tot_knowledge_eos,
                    training=False,
                    mc_token_ids=mc_token_ids
                )
                # import pdb; pdb.set_trace()
                lm_logits, knowledge_logits, persona_logits = output[0], output[1], output[2]

                persona, knowledge = 50267, 50268
                bos, padding, eos = 0, 1, 2
                device = input_ids.get_device()

                # import pdb; pdb.set_trace()
                persona_tensor = torch.tensor([persona]).cuda(device)
                knowledge_tensor = torch.tensor([knowledge]).cuda(device)
                bos_tensor = torch.tensor([bos]).cuda(device)
                eos_tensor = torch.tensor([eos]).cuda(device)
                max_position = 1024

                # import pdb; pdb.set_trace()
                sigmoid = Sigmoid()
                persona_pred_sigmoid = sigmoid(persona_logits)
                persona_pred_sigmoid = (persona_pred_sigmoid > 0.5).float()
                all_persona_pred = []
                selected_persona_idx = list()
                for batch_idx, persona_batch in enumerate(torch.eq(persona_pred_sigmoid, 1)):
                    batch_list_idx = list()
                    batch_list = list()
                    for i, can in enumerate(persona_batch):
                        if can == True:
                            batch_list_idx.append(can)
                            persona_selected_now = persona_candidates[batch_idx][i]
                            mask_persona = torch.ne(persona_selected_now, padding)
                            persona_selected_now = torch.masked_select(persona_selected_now, mask_persona)
                            batch_list.append(persona_selected_now[:-2])
                    all_persona_pred.append(batch_list)
                    selected_persona_idx.append(batch_list_idx)

                p_index_cvtd = persona_pred_sigmoid

                # import pdb; pdb.set_trace()
                softmax = Softmax(dim=-1)
                knowledge_softmax = softmax(knowledge_logits)
                _, k_index_1 = torch.topk(knowledge_softmax, k=1, dim=-1)
                all_knowledge_pred = []
                # all_k_graph_pred = []
                # all_ner_pred = []
                # all_k_graph_raw_pred = []
                # import pdb; pdb.set_trace()
                for batch_i in range(args.test_batch_size):
                    knowledge_pred_idx = k_index_1[batch_i]
                    # import pdb; pdb.set_trace()
                    knowledge_pred = knowledge_candidates[batch_i][knowledge_pred_idx]
                    # import pdb; pdb.set_trace()
                    mask_knowledge = torch.ne(knowledge_pred, padding)
                    knowledge_pred = torch.masked_select(knowledge_pred, mask_knowledge)
                    knowledge_pred = knowledge_pred[1:-2]
                    all_knowledge_pred.append(knowledge_pred) #delete bos, knowledge_st, eos

                    # import pdb; pdb.set_trace()
                    # k_graph_pred = filtered_triples[batch_i][knowledge_pred_idx]
                    # mask_k_graph = torch.ne(k_graph_pred, padding)
                    # k_graph_pred = torch.masked_select(k_graph_pred, mask_k_graph)
                    # all_k_graph_pred.append(k_graph_pred)
                    #
                    # import pdb; pdb.set_trace()
                    # ner_pred = ner_cans[batch_i][knowledge_pred_idx]
                    # mask_ner = torch.ne(ner_pred, padding)
                    # ner_pred = torch.masked_select(ner_pred, mask_ner)
                    # all_ner_pred.append(ner_pred)

                    # TODO : Any processing?
                    # all_k_graph_raw_pred.append([[tokenizer.decode(value) for value in triple] for triple in filtered_triples_raw[batch_i]])

                # import pdb; pdb.set_trace()
                # import pdb; pdb.set_trace()

                final_input_list = []
                for batch_i in range(args.test_batch_size):
                    only_dial_input_ids_batch = dialog[batch_i]
                    mask_only_dial_input_ids_batch = torch.ne(only_dial_input_ids_batch, padding)
                    only_dial_input_ids_batch = torch.masked_select(only_dial_input_ids_batch, mask_only_dial_input_ids_batch)
                    if len(all_persona_pred[batch_i])>0:
                        concat_persona = torch.cat(all_persona_pred[batch_i], dim=-1)
                        new_persona = torch.cat([persona_tensor, concat_persona], dim=-1)
                    else:
                        new_persona = None
                    new_knowledge = torch.cat([knowledge_tensor, all_knowledge_pred[batch_i]], dim=-1)

                    if new_persona is not None:
                        new_input = torch.cat([bos_tensor, new_knowledge, new_persona, only_dial_input_ids_batch, eos_tensor], dim=-1)
                    else:
                        new_input = torch.cat([bos_tensor, new_knowledge, only_dial_input_ids_batch, eos_tensor], dim=-1)
                    new_input_size = new_input.size()[0]

                    if new_input_size < int(max_position) :
                        padding_size = int(max_position) -new_input_size
                        add_padding = torch.tensor([padding]*padding_size).cuda(device)
                        final_input = torch.cat([new_input, add_padding], dim=-1)
                    final_input_list.append(final_input)
                final_input_tensor = torch.stack(final_input_list)
                decoder_input_ids = bos_tensor.unsqueeze(0)
                # print("Knowledge used : " + tokenizer.decode(all_knowledge_pred[0]))
                # TODO :
                # import pdb; pdb.set_trace()
                k_graph_ids = [tokenizer.decode(batch_triples, skip_special_tokens=True) for batch_triples in filtered_triples_raw[0]]
                k_graph_ids = [[[toks for toks in triple_rep.split(" [sep] ")] \
                                 for triple_rep in triple_reps.split(" [triple_sep] ")] for triple_reps in k_graph_ids]
                # import pdb; pdb.set_trace()
                out_ids = sample_sequence(final_input_tensor, token_type_ids=None, decoder_input_ids=decoder_input_ids,
                                          tokenizer=tokenizer, model=model, args=args, current_output=None, ner_cand_ids=[[]],
                                          knowledge_cand_ids=all_knowledge_pred, k_graph_cand_ids=[[]],
                                          k_graph_multiple_markov_triple_ids=k_graph_ids)
            else:
                raise ValueError(f"wrong model : {model.config.model_type}")

            machine, human, persona, knowledge = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)
            special_tokens_list = [machine, human, persona, knowledge, tokenizer.pad_token_id, tokenizer.bos_token_id, tokenizer.eos_token_id]

            # import pdb;pdb.set_trace()
            for special_token in special_tokens_list:
                print_dialog = [value for value in dialog[0] if value != special_token]
                print_ID = [value for value in dialog_ID[0] if value != special_token]
                print_out = [value for value in out_ids if value != special_token]
            print_ID = tokenizer.decode(print_ID, skip_special_tokens=True)
            pred_reply = tokenizer.decode(print_out, skip_special_tokens=True)

            # import pdb; pdb.set_trace()
            # This is actually performed against BART predictions. Why not keep it?
            # PG
            p_label_cvtd = torch.tensor([1 if num in persona_grounding else 0 for num in range(5)], device=args.device)
            pg.update((p_index_cvtd.squeeze(), p_label_cvtd))
            pg_res = pg.compute()

            # KG
            k_label_cvtd = torch.tensor([1 if num in knowledge_grounding else 0 for num in range(10)], device=args.device)
            kg.update((knowledge_softmax, k_label_cvtd))
            kg_res = kg.compute()

            if args.submit_path:
                # TODO : Check speed & memory?
                submit_output[print_ID].append({
                    "generation": pred_reply
                })
                # import pdb; pdb.set_trace()

            # import pdb; pdb.set_trace()
            print(f'{print_ID}, question: ', tokenizer.decode(print_dialog, skip_special_tokens=True))
            if args.no_eval:
                print('pred: ', pred_reply)
                print('---------------------------------------')
            else:
                gold_reply = reply
                #print('reply', reply)

                gold_reply = tokenizer.decode(gold_reply.tolist(), skip_special_tokens=True)
                print('gold: ', gold_reply)
                print('pred: ', pred_reply)
                print('---------------------------------------')

                #ROUGE
                r = rouge_metric.score(pred_reply, gold_reply)
                r1 += r['rouge1'].fmeasure
                r2 += r['rouge2'].fmeasure
                rl += r['rougeL'].fmeasure

                #BLEU1,2,3,4 / BLEU avg
                bleu += bleu_metric.compute(predictions=[pred_reply], references=[[gold_reply]])['score']

                if args.gen_eval:
                    # print(pred_reply)
                    # print(gold_reply)

                    #BERTScore
                    bertscore_dict = bertscore_metric.compute(predictions=[pred_reply], references=[gold_reply], lang="en")
                    bertscore_f1 += sum(bertscore_dict['f1'])
                    bertscore_precision += sum(bertscore_dict['precision'])
                    bertscore_recall += sum(bertscore_dict['recall'])

                    #BLEURT
                    # bleurt += sum(bleurt_metric.compute(predictions=[pred_reply], references=[gold_reply])["scores"])

                    #MAUVE
                    # mauve += mauve_metric.compute(predictions=[pred_reply], references=[gold_reply]).mauve

                #CharF1
                chrf1 += chrf_metric([pred_reply], [[gold_reply]])

        # Result printing out of loop
        print('test pass list: ', test_pass_list, 'len: ', pass_num)

        if not args.no_eval:
            chrf1_result = chrf1/(test_data_index+1-pass_num)
            rouge1_result = r1/(test_data_index+1-pass_num)
            rouge2_result = r2/(test_data_index+1-pass_num)
            rougel_result = rl/(test_data_index+1-pass_num)
            bleu_result = bleu/(test_data_index+1-pass_num)
            bertscore_f1_result = bertscore_f1/(test_data_index+1-pass_num)
            bertscore_precision_result = bertscore_precision/(test_data_index+1-pass_num)
            bertscore_recall_result = bertscore_recall/(test_data_index+1-pass_num)
            # bleurt_result = bleurt/(test_data_index+1-pass_num)
            # mauve_result = mauve/(test_data_index+1-pass_num)
            print("F1: ", chrf1_result)
            print("ROUGE1", rouge1_result)
            print("ROUGE2", rouge2_result)
            print("ROUGEL", rougel_result)
            print("avg BLEU: ", bleu_result)
            if args.gen_eval:
                print("BERTScore F1: ", bertscore_f1_result)
                print("BERTScore Precision: ", bertscore_precision_result)
                print("BERTScore Recall: ", bertscore_recall_result)
                # print("BLEURT", bleurt_result)
                # print("MAUVE", mauve_result)

        pg_result = pg_res
        kg_result = kg_res

        print("PG: ", pg_result)
        print("KG: ", kg_result)

        submit_path = args.submit_path
        if submit_path:
            final_out = [{id: gen} for id, gen in submit_output.items()]
            import json
            with open(submit_path, 'a') as file:
                file.write(json.dumps(final_out))


if __name__ == "__main__":
    run()

In [ ]:
# inference -> BART -> generation -> ids -> decoding -> text

In [61]:
tokenizer.convert_ids_to_tokens([1,3,5,5,7])

['[unused0]', '[unused2]', '[unused4]', '[unused4]', '[unused6]']

In [61]:
# data -> persona, dialogue, knowledge -> special tokens -> LM train -> decoding (inference)